For Single Word File

In [1]:
from docx import Document
import html
import re
import pandas as pd

# ------------------- Helpers -------------------
def is_list_item(paragraph):
    return paragraph._element.pPr is not None and paragraph._element.pPr.numPr is not None

def convert_run_to_html(run):
    text = html.escape(run.text)
    if not text:
        return ""
    if run.bold:
        text = f"<strong>{text}</strong>"
    if run.italic:
        text = f"<em>{text}</em>"
    if run.underline:
        text = f"<u>{text}</u>"
    return text

def paragraph_to_html(paragraph):
    text = "".join(convert_run_to_html(run) for run in paragraph.runs)
    if not text.strip():
        return ""
    if is_list_item(paragraph):
        return f"<li>{text}</li>"
    style = getattr(paragraph.style, "name", "").lower()
    if "heading" in style:
        level = ''.join(filter(str.isdigit, style)) or "2"
        return f"<h{level}>{text}</h{level}>"
    return f"<p>{text}</p>"

# ------------------- Extractors -------------------
def extract_title(docx_path):
    doc = Document(docx_path)
    capture = False

    for para in doc.paragraphs:
        text = para.text.strip()
        low = text.lower()

        # Detect "Report Title" subheading (numbered, bulleted, emoji, or plain)
        if not capture and ("report title" in low):
            capture = True
            continue  # skip the heading line itself

        # After capture → return first non-empty paragraph
        if capture and text:
            return text

    return ""


# ✅ Description (Introduction → stop before Report Summary/8.)
def extract_description(docx_path):
    doc = Document(docx_path)
    html_output, inside_list = [], False
    start, stop = False, False

    for para in doc.paragraphs:
        text = para.text.strip().lower()

        if not start and ("introduction" in text):
            start = True

        if start and not stop:
            html_part = paragraph_to_html(para)
            if html_part:
                if html_part.startswith("<li>"):
                    if not inside_list:
                        html_output.append("<ul>")
                        inside_list = True
                    html_output.append(html_part)
                else:
                    if inside_list:
                        html_output.append("</ul>")
                        inside_list = False
                    html_output.append(html_part)

        if text.startswith("8.") or "report summary" in text:
            stop = True
            break

    if inside_list:
        html_output.append("</ul>")
    return "\n".join(html_output)

# ✅ TOC (after Heading 9 / Table of Contents)
def extract_toc(docx_path):
    doc = Document(docx_path)
    html_output, inside_list, capture = [], False, False
    end_reached = False

    for para in doc.paragraphs:
        text = para.text.strip()
        low = text.lower()

        # Start condition
        if not capture and "table of contents" in low:
            capture = True
            continue

        if capture:
            # End condition = capture "List of Figures" + its items, then stop
            if "list of figures" in low:
                html_part = paragraph_to_html(para)
                if html_part:
                    html_output.append(html_part)   # add heading "List of Figures"
                end_reached = True
                continue  # don't break yet, because its children may follow

            if end_reached:
                # If koi aur heading/subheading mil gaya to yahan break
                style = getattr(para.style, "name", "").lower()
                if "heading" in style or re.match(r"^\d+[\.\)]\s", text):
                    break  

            html_part = paragraph_to_html(para)
            if html_part:
                if html_part.startswith("<li>"):
                    if not inside_list:
                        html_output.append("<ul>")
                        inside_list = True
                    html_output.append(html_part)
                else:
                    if inside_list:
                        html_output.append("</ul>")
                        inside_list = False
                    html_output.append(html_part)

    if inside_list:
        html_output.append("</ul>")
    return "\n".join(html_output)


# ✅ extract description (Heading 1 → Heading 7 only)

def extract_description(docx_path):
    doc = Document(docx_path)
    html_output = []
    inside_list = False
    capture = False  

    for para in doc.paragraphs:
        text = para.text.strip()
        if not text:
            continue
        low = text.lower()

        # --- Start Condition ---
        if not capture and "introduction and strategic context" in low:
            capture = True  

        # --- Stop Condition ---
        if capture and "report summary, faqs, and seo schema" in low:
            break  

        if capture:
            html_part = paragraph_to_html(para)

            if not html_part:
                continue

            if html_part.startswith("<li>"):
                if not inside_list:
                    html_output.append("<ul>")
                    inside_list = True
                html_output.append(html_part)
            else:
                if inside_list:
                    html_output.append("</ul>")
                    inside_list = False
                html_output.append(html_part)

    if inside_list:
        html_output.append("</ul>")

    return "\n".join(html_output)

# ✅ Methodology (FAQ Table or Paragraphs Qn:/A: format)
def extract_methodology(docx_path):
    doc = Document(docx_path)
    faqs, q_count = [], 0

    # --- Case 1: FAQ Table ---
    for table in doc.tables:
        headers = [cell.text.strip().lower() for cell in table.rows[0].cells]
        if "question" in headers and "answer" in headers:
            for row in table.rows[1:]:
                q_text = row.cells[0].text.strip()
                a_text = row.cells[1].text.strip()
                if q_text and a_text:
                    q_count += 1
                    faqs.append(
                        f"<p><strong>Q{q_count}: {html.escape(q_text)}</strong><br>"
                        f"A{q_count}: {html.escape(a_text)}</p>"
                    )
            if faqs:
                return "\n".join(faqs)  # return if table found

    # --- Case 2 & 3: Text-based FAQs ---
    capture = False
    for para in doc.paragraphs:
        text = para.text.strip()
        if not text:
            continue

        # Start after "Top 5 FAQs" heading
        if "top 5 faqs" in text.lower():
            capture = True
            continue

        if capture:
            # ---- Case 3: Q and A in same paragraph ----
            both_match = re.match(r"Q\d+[:.]\s*(.*?)\s*A\d*[:.]\s*(.*)", text, re.IGNORECASE)
            if both_match:
                q_count += 1
                faqs.append(
                    f"<p><strong>Q{q_count}: {html.escape(both_match.group(1).strip())}</strong><br>"
                    f"A{q_count}: {html.escape(both_match.group(2).strip())}</p>"
                )
                continue

            # ---- Case 2: Q... in one para, A... in next ----
            q_match = re.match(r"Q\d+[:.]\s*(.*)", text, re.IGNORECASE)
            if q_match:
                current_q = q_match.group(1).strip()
                continue

            a_match = re.match(r"A\d*[:.]\s*(.*)", text, re.IGNORECASE)
            if a_match and 'current_q' in locals():
                q_count += 1
                faqs.append(
                    f"<p><strong>Q{q_count}: {html.escape(current_q)}</strong><br>"
                    f"A{q_count}: {html.escape(a_match.group(1).strip())}</p>"
                )
                del current_q  # reset

    return "\n".join(faqs)



# ✅ Meta Description (first para after Introduction)
def extract_meta_description(docx_path):
    doc = Document(docx_path)
    capture = False
    for para in doc.paragraphs:
        text = para.text.strip()
        low = text.lower()
        if not capture and ("introduction" in low):
            capture = True
            continue
        if capture and text:
            return text
    return ""

# ✅ Detect new subheading (for SeoTitle/BreadcrumbText)
def _is_new_subheading(text: str) -> bool:
    text = text.strip()
    if not text:
        return False
    if text.startswith(("📊", "❓", "🧩")):
        return True
    if re.match(r"^[A-Z](?:\.\d+)?\.\s", text):
        return True
    if re.match(r"^\d+\.\s", text):
        return True
    return False

# ✅ SeoTitle & BreadcrumbText (A.3. Headline block)
def extract_seo_title_and_breadcrumb(docx_path):
    doc = Document(docx_path)
    capture, lines = False, []
    for para in doc.paragraphs:
        text = para.text.strip()
        low = text.lower()
        if not capture and ("a.3. headline" in low or low.startswith("headline")):
            capture = True
            continue
        if capture:
            if _is_new_subheading(text):
                break
            if text:
                lines.append(text)
                if len(lines) >= 2:
                    break
    seo_title = lines[0] if len(lines) >= 1 else ""
    breadcrumb_text = lines[1] if len(lines) >= 2 else ""
    return seo_title, breadcrumb_text

# ------------------- Runner -------------------

doc_path = r"C:\Users\Vishnu\Documents\extracted_docs\extracted_docs\Aerial Refueling System Market.docx"
title=extract_title(doc_path)
description_html = extract_description(doc_path)
toc_html = extract_toc(doc_path)
methodology_html = extract_methodology(doc_path)
meta = extract_meta_description(doc_path)
seo_title, breadcrumb_text = extract_seo_title_and_breadcrumb(doc_path)

# ✅ Save single file result in Excel
df = pd.DataFrame([{
    "Title":title,
    "Description": description_html,
    "TOC": toc_html,
    "Methodology": methodology_html,
    "Meta Description": meta,
    "SeoTitle": seo_title,
    "BreadcrumbText": breadcrumb_text
}])

output_path = r"C:\Users\Vishnu\Documents\extracted_docs\SingleFile.xlsx"
df.to_excel(output_path, index=False)

print(f"✅ Done! Extracted data saved in {output_path}")


✅ Done! Extracted data saved in C:\Users\Vishnu\Documents\extracted_docs\SingleFile.xlsx


For Multiple Word File


In [ ]:
from docx import Document
import re
import os
import pandas as pd
from datetime import date
# ------------------- Helpers -------------------
DASH = "–"  # en-dash for year ranges

HEADER_LINE_RE = re.compile(
    r"""^\s*
        (?:[A-Za-z]\.)?
        (?:\d+(?:\.\d+)*)?
        [\.\)]?\s*
        (?:report\s*title|full\s*title|full\s*report\s*title|title\s*\(long[-\s]*form\))
        [\s:–-]*$
    """, re.I | re.X
)

def _inline_title(text: str) -> str:
    m = re.split(r"[:\-–]", text, maxsplit=1)
    if len(m) > 1:
        right = m[1].strip()
        if right and not HEADER_LINE_RE.match(right):
            return right
    return ""

def _year_range_present(text: str) -> bool:
    return bool(re.search(r"20\d{2}\s*[\-–]\s*20\d{2}", text))

def _ensure_filename_start_and_year(title: str, filename: str) -> str:
    if not title.lower().startswith(filename.lower()):
        title = f"{filename} {title}"
    if not _year_range_present(title):
        title = f"{title} {DASH}2024–2030"
    return _norm(title)

def _remove_emojis(text: str) -> str:
    emoji_pattern = re.compile(
        "[" "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251" "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub("", text)

def _norm(s: str) -> str:
    s = _remove_emojis(s or "")
    return re.sub(r"\s+", " ", s.strip())

# ------------------- Convert Paragraph to HTML -------------------
def paragraph_to_html(para):
    """Convert a docx paragraph into HTML with basic formatting."""
    text = para.text.strip()
    if not text:
        return ""

    # Check if it's a list item
    if para.style.name.lower().startswith("list"):
        return f"<li>{text}</li>"
    
    text = _remove_emojis(text)

    # Headings
    if para.style.name.startswith("Heading"):
        level = para.style.name.replace("Heading", "").strip()
        level = int(level) if level.isdigit() else 2
        return f"<h{level}>{text}</h{level}>"

    # Normal paragraph
    return f"<p>{text}</p>"
# ----------------------------------------------------Extract Title----------------------------
def extract_title(docx_path: str) -> str:
    from docx import Document
    import os, re

    doc = Document(docx_path)
    filename = os.path.splitext(os.path.basename(docx_path))[0]
    filename_low = filename.lower()

    blocks = [(p, (p.text or "").strip()) for p in doc.paragraphs if (p.text or "").strip()]

    capture = False
    # --- Priority 1: Explicit "Report Title" / "Long-Form" ---
    for _, text in blocks:
        if capture:
            return _ensure_filename_start_and_year(text, filename)
        if HEADER_LINE_RE.match(text):
            inline = _inline_title(text)
            if inline:
                return _ensure_filename_start_and_year(inline, filename)
            capture = True
            continue

    # --- Priority 2: Tables ---
    for table in doc.tables:
        for r_idx, row in enumerate(table.rows):
            for c_idx, cell in enumerate(row.cells):
                cell_text = (cell.text or "").strip().lower()
                if not cell_text:
                    continue
                if "report title" in cell_text or "full title" in cell_text or "full report title" in cell_text:
                    if c_idx + 1 < len(row.cells):
                        nxt = row.cells[c_idx+1].text.strip()
                        if nxt:
                            return _ensure_filename_start_and_year(nxt, filename)
                    if r_idx + 1 < len(table.rows):
                        nxt = table.rows[r_idx+1].cells[c_idx].text.strip()
                        if nxt:
                            return _ensure_filename_start_and_year(nxt, filename)
                        if c_idx + 1 < len(table.rows[r_idx+1].cells):
                            nxt2 = table.rows[r_idx+1].cells[c_idx+1].text.strip()
                            if nxt2:
                                return _ensure_filename_start_and_year(nxt2, filename)

    # --- Priority 3: "Full Title" inline variations ---
    for _, text in blocks:
        low = text.lower()
        if low.startswith("full report title") or low.startswith("full title"):
            inline = _inline_title(text)
            if inline:
                return _ensure_filename_start_and_year(inline, filename)

    # --- Priority 4: Fallback (only if nothing above matched) ---
    skip_words = ("strategically", "introduction", "context")
    for _, t in blocks:
        tt = _norm(t)
        low = tt.lower()
        if any(sw in low for sw in skip_words):
            continue
        if low.startswith("the global"):
            continue
        if "table of contents" in low or "toc" in low:
            continue
        if "list of tables" in low or "list of figures" in low:
            continue
        if _year_range_present(tt) and any(word in low for word in filename_low.split()):
            return _ensure_filename_start_and_year(tt, filename)

    return "Title Not Available"

# ------------------- Extract Description -------------------
def extract_description(docx_path):
    from docx import Document
    doc = Document(docx_path)
    html_output = []
    capture, inside_list = False, False  

    def run_to_html(run):
        """Convert bold/italic text to HTML span."""
        text = run.text.strip()
        if not text:
            return ""
        if run.bold and run.italic:
            return f"<b><i>{text}</i></b>"
        elif run.bold:
            return f"<b>{text}</b>"
        elif run.italic:
            return f"<i>{text}</i>"
        return text

    for para in doc.paragraphs:
        text = para.text.strip()
        if not text:
            continue
        low = text.lower()

        # --- Start condition ---
        if not capture and (
            "introduction and strategic context" in low
            or text.startswith("the global")
            or "market will witness" in low
        ):
            capture = True
            html_output.append("<h2>Introduction and Strategic Context</h2>")

        # --- Stop condition ---
        if capture and "report summary, faqs, and seo schema" in low:
            break  

        if capture:
            # Convert runs (bold/italic parts) to inline HTML
            content = "".join(run_to_html(run) for run in para.runs if run.text.strip())

            # Heading handling
            if para.style.name.startswith("Heading"):
                level = para.style.name.replace("Heading", "").strip()
                level = int(level) if level.isdigit() else 2
                html_output.append(f"<h{level}>{content}</h{level}>")
                continue

            # List handling
            if "list" in para.style.name.lower():
                if not inside_list:
                    html_output.append("<ul>")
                    inside_list = True
                html_output.append(f"<li>{content}</li>")
                continue
            else:
                if inside_list:
                    html_output.append("</ul>")
                    inside_list = False

            # Normal paragraph
            html_output.append(f"<p>{content}</p>")

    if inside_list:
        html_output.append("</ul>")
    return _remove_emojis("\n".join(html_output))

# -------------------------------TOC (after Heading 9 / Table of Contents)--------------------------------------
def extract_toc(docx_path):
    doc = Document(docx_path)
    html_output, inside_list, capture = [], False, False
    end_reached = False

    for para in doc.paragraphs:
        text = para.text.strip()
        low = text.lower()

        # Start condition
        if not capture and "table of contents" in low:
            capture = True
            continue

        if capture:
            # End condition = capture "List of Figures" + its items, then stop
            if "list of figures" in low:
                html_part = paragraph_to_html(para)
                if html_part:
                    html_output.append(html_part)   # add heading "List of Figures"
                end_reached = True
                continue  # don't break yet, because its children may follow

            if end_reached:
                style = getattr(para.style, "name","").lower()
                if "heading" in style or re.match(r"^\d+[\.\)]\s", text):
                    break  

            html_part = paragraph_to_html(para)
            if html_part:
                if html_part.startswith("<li>"):
                    if not inside_list:
                        html_output.append("<ul>")
                        inside_list = True
                    html_output.append(html_part)
                else:
                    if inside_list:
                        html_output.append("</ul>")
                        inside_list = False
                    html_output.append(html_part)

    if inside_list:
        html_output.append("</ul>")
    return _remove_emojis("".join(html_output).strip())

# ---------------------------------------Meta Discription---------------------------------------
def extract_meta_description(docx_path):
    doc = Document(docx_path)
    capture = False
    for para in doc.paragraphs:
        text = para.text.strip()
        low = text.lower()
        if not capture and ("introduction" in low):
            capture = True
            continue
        if capture and text:
            return text
    return ""
# --------------------------------------------SEO title-------------------------------------------------------------
def extract_seo_title(docx_path):
    doc = Document(docx_path)
    file_name = os.path.splitext(os.path.basename(docx_path))[0]  # File name without extension
    
    revenue_forecast = ""

    # --- Check tables for Report Coverage ---
    for table in doc.tables:
        headers = [cell.text.strip().lower() for cell in table.rows[0].cells]

        if "report attribute" in headers and "details" in headers:
            attr_idx = headers.index("report attribute")
            details_idx = headers.index("details")

            for row in table.rows[1:]:
                attr = row.cells[attr_idx].text.strip().lower()
                details = row.cells[details_idx].text.strip()

                if "revenue forecast in 2030" in attr:
                    # replace USD with $
                    revenue_forecast = details.replace("USD", "$").strip()
                    break

    if revenue_forecast:
        seo_title = f"{file_name} Size ({revenue_forecast}) 2030"
    else:
        seo_title = file_name  # fallback

    return seo_title
# -----------------------------------------------BreadCrumb Text----------------------------------------
def extract_breadcrumb_text(docx_path):
    doc = Document(docx_path)
    file_name = os.path.splitext(os.path.basename(docx_path))[0]  # File name without extension
    
    revenue_forecast = ""

    # --- Check tables for Report Coverage ---
    for table in doc.tables:
        headers = [cell.text.strip().lower() for cell in table.rows[0].cells]

        if "report attribute" in headers and "details" in headers:
            attr_idx = headers.index("report attribute")
            details_idx = headers.index("details")

            for row in table.rows[1:]:
                attr = row.cells[attr_idx].text.strip().lower()
                details = row.cells[details_idx].text.strip()

                if "revenue forecast in 2030" in attr:
                    # replace USD with $
                    revenue_forecast = details.replace("USD", "$").strip()
                    break

    if revenue_forecast:
        seo_title = f"{file_name} Report 2030"
    else:
        seo_title = file_name  # fallback

    return seo_title

# ---------------------------------------------SkuCode-Extraction------------------------------
def extract_sku_code(docx_path):
    filename = os.path.basename(docx_path)
    sku_code = os.path.splitext(filename)[0].lower()
    return sku_code
# ---------------------------------------------URLRP------------------------------
def extract_sku_url(docx_path):
    filename = os.path.basename(docx_path)
    sku_code = os.path.splitext(filename)[0].lower()
    return sku_code

# ---------------------------------------------BreadCrumb Schema----------------------------
def extract_breadcrumb_schema(docx_path):
    doc = Document(docx_path)
    paragraphs = [p.text.strip() for p in doc.paragraphs if p.text.strip()]

    capture = False
    breadcrumb_data = []

    for text in paragraphs:
        low = text.lower()

        # Start condition → JSON block must start with {
        if not capture and text.strip().startswith("{"):
            capture = True
        
        # End condition → stop when json copy or faq schema heading found
        if capture and ("json copy" in low or "faq schema" in low):
            break

        # Collect JSON block only
        if capture:
            breadcrumb_data.append(text)

    return "".join(breadcrumb_data).strip()
# --------------------------------Schema 2-----------------------
def _get_text(docx_path):
    doc = Document(docx_path)
    return "\n".join(p.text for p in doc.paragraphs if p.text and p.text.strip())

def _extract_json_block(text, type_name):
    # find "@type": "BreadcrumbList" OR "@type": "FAQPage"
    pat = re.compile(r'"@type"\s*:\s*"' + re.escape(type_name) + r'"')
    m = pat.search(text)
    if not m:
        return ""
    start_idx = text.rfind("{", 0, m.start())
    if start_idx == -1:
        return ""
    depth, i, n = 0, start_idx, len(text)
    block_chars = []
    while i < n:
        ch = text[i]
        block_chars.append(ch)
        if ch == "{":
            depth += 1
        elif ch == "}":
            depth -= 1
            if depth == 0:
                break
        i += 1
    return "".join(block_chars).strip()

def extract_breadcrumb_schema(docx_path):
    text = _get_text(docx_path)
    return _extract_json_block(text, "BreadcrumbList")

def extract_faq_schema(docx_path):
    text = _get_text(docx_path)
    return _extract_json_block(text, "FAQPage")
# -----------------------------------------------------------------
import json
import html
def extract_methodology_from_faqschema(docx_path):
    faq_schema_str = extract_faq_schema(docx_path)  
    if not faq_schema_str:
        return ""   # agar schema na mile to blank
    
    try:
        faq_data = json.loads(faq_schema_str)
    except json.JSONDecodeError:
        return ""   # agar JSON format galat hai
    
    faqs = []
    q_count = 0
    for item in faq_data.get("mainEntity", []):
        q_count += 1
        question = item.get("name", "").strip()
        answer = item.get("acceptedAnswer", {}).get("text", "").strip()
        if question and answer:
            faqs.append(
                f"<p><strong>Q{q_count}: {html.escape(question)}</strong><br>"
                f"A{q_count}: {html.escape(answer)}</p>"
            )
    
    return "\n".join(faqs)
# --------------------------------------------Report Coverage Table-----------------------------------
def extract_report_coverage_table_with_style(docx_path):
    doc = Document(docx_path)

    for table in doc.tables:
        # check if this is Report Coverage Table
        first_row_text = " ".join([c.text.strip().lower() for c in table.rows[0].cells])
        if "report attribute" in first_row_text or "report coverage table" in first_row_text:
            html = []
            # --- Add heading ---
            html.append('<h2><strong>7.1. Report Coverage Table</strong></h2>')
            html.append('<table cellspacing="0" style="border-collapse:collapse; width:100%"><tbody>')

            for r_idx, row in enumerate(table.rows):
                html.append("<tr>")
                for c_idx, cell in enumerate(row.cells):
                    text = cell.text.strip()

                    # alternate row background
                    bg = "#deeaf6" if r_idx % 2 == 1 else "#ffffff"
                    # header row style
                    if r_idx == 0:
                        bg = "#5b9bd5"

                    # td style
                    td_style = (
                        f"background-color:{bg}; "
                        "border:1px solid #9cc2e5; vertical-align:top; padding:4px;"
                        "width:263px" if c_idx == 0 else
                        f"background-color:{bg}; border:1px solid #9cc2e5; vertical-align:top; padding:4px; width:303px"
                    )

                    # wrap text inside <p>
                    html.append(f'<td style="{td_style}"><p><strong>{text}</strong></p></td>' if c_idx == 0 or r_idx==0 else f'<td style="{td_style}"><p>{text}</p></td>')
                html.append("</tr>")
            html.append("</tbody></table>")
            return "\n".join(html)

    return ""
# ------------------------------------------------------Merger--------------------------------------------------
def merge_description_and_coverage(docx_path):
    try:
        desc_html = extract_description(docx_path) or ""
        coverage_html = extract_report_coverage_table_with_style(docx_path) or ""
        merged_html = desc_html + "\n\n" + coverage_html if (desc_html or coverage_html) else ""
        return merged_html
    except Exception as e:
        return f"ERROR: {e}"


# ------------------- Run Extraction -------------------
folder_path = r"C:\Users\Vishnu\Desktop\oldcontent\23 june\23 june"
output_path = r"C:\Users\Vishnu\Documents\extracted_docs\Extraction_New_Title_Old.xlsx"

all_data = []

for file in os.listdir(folder_path):
    if not file.endswith(".docx") or file.startswith("~$"):
        continue

    doc_path = os.path.join(folder_path, file)
    print(f"Processing: {file}")
    title = extract_title(doc_path)
    description_html = extract_description(doc_path)
    toc=extract_toc(doc_path)
    methodology=extract_methodology_from_faqschema(doc_path)
    # methodology_html = extract_methodology(doc_path)
    seo_title = extract_seo_title(doc_path)
    breadcrumb_text = extract_breadcrumb_text(doc_path)
    skucode = extract_sku_code(doc_path)
    urlrp = extract_sku_url(doc_path)
    breadcrumb_schema=extract_breadcrumb_schema(doc_path)
    meta=extract_meta_description(doc_path)
    schema2=extract_faq_schema(doc_path)
    report=extract_report_coverage_table_with_style(doc_path)
    merge=merge_description_and_coverage(doc_path)

    all_data.append({
        "File": file,
        "Title": title,
        "Description": description_html,
        "TOC":toc,
        "Segmentation":"<p>.</p>",
        "Methodology":methodology,
        "Publish_Date":date.today().strftime("%B %Y"),
        "Currency":"USD",
        "Single Price": 4485,
        "Corporate Price": 6449,
        "skucode": skucode,
        "Total Page":"",
        "Date": date.today().strftime("%Y-%m-%d"),  # always today's date
        "urlNp": urlrp,
        "Meta Discription":meta,
        "Meta Keys":"",
        "Base Year":"2024",
        "history":"2019-2023",
        "Enterprise Price": 8339,
        "SEOTITLE": seo_title,
        "BreadCrumb Text": breadcrumb_text,
        "Schema 1":breadcrumb_schema,
        "Schema 2":schema2,
        "Report":report,
        "Discription":merge
    })

df = pd.DataFrame(all_data)
df.to_excel(output_path, index=False)
print(f"Done! Extracted data saved in {output_path}")

If Cell Limit is Crossed in extract_title

In [ ]:
from docx import Document
import re
import os
import pandas as pd
from datetime import date
import json
import html

# ------------------- Helpers -------------------
DASH = "–"  # en-dash for year ranges
EXCEL_CELL_LIMIT = 32767  # Excel max char limit per cell

def split_into_excel_cells(text, limit=EXCEL_CELL_LIMIT):
    """Split text into chunks that fit within Excel cell limit."""
    if not text:
        return [""]
    return [text[i:i+limit] for i in range(0, len(text), limit)]

HEADER_LINE_RE = re.compile(
    r"""^\s*
        (?:[A-Za-z]\.)?
        (?:\d+(?:\.\d+)*)?
        [\.\)]?\s*
        (?:report\s*title|full\s*title|full\s*report\s*title|title\s*\(long[-\s]*form\))
        [\s:–-]*$
    """, re.I | re.X
)

def _inline_title(text: str) -> str:
    m = re.split(r"[:\-–]", text, maxsplit=1)
    if len(m) > 1:
        right = m[1].strip()
        if right and not HEADER_LINE_RE.match(right):
            return right
    return ""

def _year_range_present(text: str) -> bool:
    return bool(re.search(r"20\d{2}\s*[\-–]\s*20\d{2}", text))

def _ensure_filename_start_and_year(title: str, filename: str) -> str:
    if not title.lower().startswith(filename.lower()):
        title = f"{filename} {title}"
    if not _year_range_present(title):
        title = f"{title} {DASH}2024–2030"
    return _norm(title)

def _remove_emojis(text: str) -> str:
    emoji_pattern = re.compile(
        "[" "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251" "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub("", text)

def _norm(s: str) -> str:
    s = _remove_emojis(s or "")
    return re.sub(r"\s+", " ", s.strip())

# ------------------- Convert Paragraph to HTML -------------------
def paragraph_to_html(para):
    text = para.text.strip()
    if not text:
        return ""

    if para.style.name.lower().startswith("list"):
        return f"<li>{text}</li>"

    text = _remove_emojis(text)

    if para.style.name.startswith("Heading"):
        level = para.style.name.replace("Heading", "").strip()
        level = int(level) if level.isdigit() else 2
        return f"<h{level}>{text}</h{level}>"

    return f"<p>{text}</p>"

# ----------------------------------------------------Extract Title----------------------------
def extract_title(docx_path: str) -> str:
    doc = Document(docx_path)
    filename = os.path.splitext(os.path.basename(docx_path))[0]
    filename_low = filename.lower()
    blocks = [(p, (p.text or "").strip()) for p in doc.paragraphs if (p.text or "").strip()]

    capture = False
    for _, text in blocks:
        if capture:
            return _ensure_filename_start_and_year(text, filename)
        if HEADER_LINE_RE.match(text):
            inline = _inline_title(text)
            if inline:
                return _ensure_filename_start_and_year(inline, filename)
            capture = True
            continue

    for table in doc.tables:
        for r_idx, row in enumerate(table.rows):
            for c_idx, cell in enumerate(row.cells):
                cell_text = (cell.text or "").strip().lower()
                if not cell_text:
                    continue
                if "report title" in cell_text or "full title" in cell_text or "full report title" in cell_text:
                    if c_idx + 1 < len(row.cells):
                        nxt = row.cells[c_idx+1].text.strip()
                        if nxt:
                            return _ensure_filename_start_and_year(nxt, filename)
                    if r_idx + 1 < len(table.rows):
                        nxt = table.rows[r_idx+1].cells[c_idx].text.strip()
                        if nxt:
                            return _ensure_filename_start_and_year(nxt, filename)
                        if c_idx + 1 < len(table.rows[r_idx+1].cells):
                            nxt2 = table.rows[r_idx+1].cells[c_idx+1].text.strip()
                            if nxt2:
                                return _ensure_filename_start_and_year(nxt2, filename)

    for _, text in blocks:
        low = text.lower()
        if low.startswith("full report title") or low.startswith("full title"):
            inline = _inline_title(text)
            if inline:
                return _ensure_filename_start_and_year(inline, filename)

    skip_words = ("strategically", "introduction", "context")
    for _, t in blocks:
        tt = _norm(t)
        low = tt.lower()
        if any(sw in low for sw in skip_words):
            continue
        if low.startswith("the global"):
            continue
        if "table of contents" in low or "toc" in low:
            continue
        if "list of tables" in low or "list of figures" in low:
            continue
        if _year_range_present(tt) and any(word in low for word in filename_low.split()):
            return _ensure_filename_start_and_year(tt, filename)

    return "Title Not Available"

# ------------------- Extract Description -------------------
def extract_description(docx_path):
    doc = Document(docx_path)
    html_output = []
    capture, inside_list = False, False  

    def run_to_html(run):
        text = run.text.strip()
        if not text:
            return ""
        if run.bold and run.italic:
            return f"<b><i>{text}</i></b>"
        elif run.bold:
            return f"<b>{text}</b>"
        elif run.italic:
            return f"<i>{text}</i>"
        return text

    for para in doc.paragraphs:
        text = para.text.strip()
        if not text:
            continue
        low = text.lower()

        if not capture and (
            "introduction and strategic context" in low
            or text.startswith("the global")
            or "market will witness" in low
        ):
            capture = True
            html_output.append("<h2>Introduction and Strategic Context</h2>")

        if capture and "report summary, faqs, and seo schema" in low:
            break  

        if capture:
            content = "".join(run_to_html(run) for run in para.runs if run.text.strip())
            if para.style.name.startswith("Heading"):
                level = para.style.name.replace("Heading", "").strip()
                level = int(level) if level.isdigit() else 2
                html_output.append(f"<h{level}>{content}</h{level}>")
                continue

            if "list" in para.style.name.lower():
                if not inside_list:
                    html_output.append("<ul>")
                    inside_list = True
                html_output.append(f"<li>{content}</li>")
                continue
            else:
                if inside_list:
                    html_output.append("</ul>")
                    inside_list = False

            html_output.append(f"<p>{content}</p>")

    if inside_list:
        html_output.append("</ul>")
    return _remove_emojis("\n".join(html_output))

# ------------------- Coverage Table -------------------
def extract_report_coverage_table_with_style(docx_path):
    doc = Document(docx_path)
    for table in doc.tables:
        first_row_text = " ".join([c.text.strip().lower() for c in table.rows[0].cells])
        if "report attribute" in first_row_text or "report coverage table" in first_row_text:
            html = []
            html.append('<h2><strong>7.1. Report Coverage Table</strong></h2>')
            html.append('<table cellspacing="0" style="border-collapse:collapse; width:100%"><tbody>')
            for r_idx, row in enumerate(table.rows):
                html.append("<tr>")
                for c_idx, cell in enumerate(row.cells):
                    text = cell.text.strip()
                    bg = "#deeaf6" if r_idx % 2 == 1 else "#ffffff"
                    if r_idx == 0:
                        bg = "#5b9bd5"
                    td_style = (
                        f"background-color:{bg}; border:1px solid #9cc2e5; vertical-align:top; padding:4px; width:263px"
                        if c_idx == 0 else
                        f"background-color:{bg}; border:1px solid #9cc2e5; vertical-align:top; padding:4px; width:303px"
                    )
                    html.append(
                        f'<td style="{td_style}"><p><strong>{text}</strong></p></td>'
                        if c_idx == 0 or r_idx == 0 else f'<td style="{td_style}"><p>{text}</p></td>'
                    )
                html.append("</tr>")
            html.append("</tbody></table>")
            return "\n".join(html)
    return ""

# ------------------- Merger -------------------
def merge_description_and_coverage(docx_path):
    try:
        desc_html = extract_description(docx_path) or ""
        coverage_html = extract_report_coverage_table_with_style(docx_path) or ""
        merged_html = desc_html + "\n\n" + coverage_html if (desc_html or coverage_html) else ""
        return merged_html
    except Exception as e:
        return f"ERROR: {e}"

# ------------------- Run Extraction -------------------
folder_path = r"C:\Users\Vishnu\Desktop\oldcontent\23 june\23 june"
output_path = r"C:\Users\Vishnu\Documents\extracted_docs\Extraction_New_Title_Old.xlsx"

all_data = []

for file in os.listdir(folder_path):
    if not file.endswith(".docx") or file.startswith("~$"):
        continue

    doc_path = os.path.join(folder_path, file)
    print(f"Processing: {file}")

    title = extract_title(doc_path)
    description_html = extract_description(doc_path)
    report = extract_report_coverage_table_with_style(doc_path)
    merge = merge_description_and_coverage(doc_path)

    # Split merged description into Excel chunks
    chunks = split_into_excel_cells(merge)

    row_data = {
        "File": file,
        "Title": title,
        "Description": description_html,
        "Report": report,
    }

    for i, chunk in enumerate(chunks, start=1):
        row_data[f"Discription_Part{i}"] = chunk

    all_data.append(row_data)

df = pd.DataFrame(all_data)
df.to_excel(output_path, index=False)
print(f"Done! Extracted data saved in {output_path}")
